In [ ]:
import numpy as np
import cv2
from pathlib import Path

In [ ]:
# === 1) Hàm load dữ liệu .npz ===
def load_npz_data(npz_path: Path):
    """Đọc u, v (pixel coords) và cam_xyz (Nx3) từ file .npz."""
    data = np.load(npz_path)
    return data['u'], data['v'], data['cam_xyz']

In [ ]:
# === 2) Tạo depth map 2D (float32) ===
def build_depth_map(u: np.ndarray,
                    v: np.ndarray,
                    cam_xyz: np.ndarray,
                    H: int,
                    W: int,
                    fill_value: float = np.inf) -> np.ndarray:
    """
    Rasterize các điểm 3D xuống ảnh (H,W), giữ giá trị depth nhỏ nhất
    nếu nhiều điểm cùng rơi vào 1 pixel.
    fill_value nên là np.inf để clamp dễ hơn.
    """
    dists = np.linalg.norm(cam_xyz, axis=1)
    depth = np.full((H, W), fill_value, dtype=np.float32)

    for ui, vi, di in zip(u.astype(int), v.astype(int), dists):
        if 0 <= ui < W and 0 <= vi < H:
            # giữ depth gần nhất
            if di < depth[vi, ui]:
                depth[vi, ui] = di

    # pixel chưa gán sẽ vẫn = np.inf, đặt thành 0
    depth[depth == np.inf] = 0.0
    return depth

In [ ]:
# === 3) Chuẩn hóa lên 16-bit và clamp 100m ===
def normalize_depth(depth_map: np.ndarray,
                       max_dist: float = 100.0) -> np.ndarray:
    """
    - Clip depth về [0, max_dist]
    - Scale về [0 .. 65535], dtype=uint16
    """
    # 3.1 clip
    depth_clipped = np.clip(depth_map, 0.0, max_dist)
    # 3.2 scale
    depth_16 = np.round(depth_clipped / max_dist * 65535.0).astype(np.uint16)
    return depth_16

In [ ]:
# === 4) Hàm tạo dataset RGB-D (4-ch) ===
def create_rgbd_dataset(
    image_dir: Path,
    npz_dir: Path,
    output_dir: Path,
    image_ext: str = ".png",
    max_dist: float = 100.0
):
    """
    Với mỗi ảnh RGB <stem>.png và file điểm <stem>_data.npz,
    sinh ảnh 4-kênh <stem>_rgbd.png (B,G,R,depth16).
    """
    output_dir.mkdir(parents=True, exist_ok=True)
    for img_path in sorted(image_dir.glob(f"*{image_ext}")):
        stem     = img_path.stem
        npz_path = npz_dir / f"{stem}_data.npz"
        if not npz_path.exists():
            print(f"[!] Thiếu NPZ: {npz_path}, bỏ qua {stem}")
            continue

        # 1) Load RGB và point-cloud
        img       = cv2.imread(str(img_path), cv2.IMREAD_COLOR)  # BGR uint8
        H, W      = img.shape[:2]
        u, v, cz  = load_npz_data(npz_path)

        # 2) Build depth map float32
        depth_map = build_depth_map(u, v, cz, H, W, fill_value=np.inf)

        # 3) Normalize & quantize depth → uint16
        depth16   = normalize_depth(depth_map, max_dist=max_dist)

        # 4) Stack BGR(uint8) → uint16 và depth16
        bgr16     = img.astype(np.uint16)
        rgbd16    = np.dstack((bgr16, depth16))  # shape (H, W, 4), dtype=uint16

        # 5) Lưu PNG 16-bit, file vẫn là <stem>_rgbd.png
        out_path  = output_dir / f"{stem}_rgbd.png"
        cv2.imwrite(str(out_path), rgbd16)
        print(f"[✓] Saved RGB-D 16-bit: {out_path}")

In [ ]:
# === 5) Chạy cho train & test ===
if __name__ == "__main__":
    DATA_ROOT     = Path("/content/drive/MyDrive/Khóa Luận/Data_Object")
    # train
    IMG_TRAIN     = DATA_ROOT / "training/image_2"
    NPZ_TRAIN     = DATA_ROOT / "output_npz_training"
    OUT_RGBD_TRAIN= DATA_ROOT / "Code/Early_Fusion_RGBD_Yolov8/4chan/rgbd_images_training"
    # test
    IMG_TEST      = DATA_ROOT / "testing/image_2"
    NPZ_TEST      = DATA_ROOT / "output_npz_testing"
    OUT_RGBD_TEST = DATA_ROOT / "Code/Early_Fusion_RGBD_Yolov8/4chan/rgbd_images_testing"

    create_rgbd_dataset(IMG_TRAIN, NPZ_TRAIN, OUT_RGBD_TRAIN, max_dist=100.0)
    create_rgbd_dataset(IMG_TEST,  NPZ_TEST,  OUT_RGBD_TEST,  max_dist=100.0)

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
[✓] Saved RGB-D 16-bit: /content/drive/MyDrive/Khóa Luận/Data_Object/Code/Early_Fusion_RGBD_Yolov8/4chan/rgbd_images_testing/002518_rgbd.png
[✓] Saved RGB-D 16-bit: /content/drive/MyDrive/Khóa Luận/Data_Object/Code/Early_Fusion_RGBD_Yolov8/4chan/rgbd_images_testing/002519_rgbd.png
[✓] Saved RGB-D 16-bit: /content/drive/MyDrive/Khóa Luận/Data_Object/Code/Early_Fusion_RGBD_Yolov8/4chan/rgbd_images_testing/002520_rgbd.png
[✓] Saved RGB-D 16-bit: /content/drive/MyDrive/Khóa Luận/Data_Object/Code/Early_Fusion_RGBD_Yolov8/4chan/rgbd_images_testing/002521_rgbd.png
[✓] Saved RGB-D 16-bit: /content/drive/MyDrive/Khóa Luận/Data_Object/Code/Early_Fusion_RGBD_Yolov8/4chan/rgbd_images_testing/002522_rgbd.png
[✓] Saved RGB-D 16-bit: /content/drive/MyDrive/Khóa Luận/Data_Object/Code/Early_Fusion_RGBD_Yolov8/4chan/rgbd_images_testing/002523_rgbd.png
[✓] Saved RGB-D 16-bit: /content/drive/MyDrive/Khóa Luận/Data_Object/Code/Early_Fusion_RGBD_Yolov

# **create img RGBD with D using inpaint**

In [1]:
import numpy as np
import cv2
from pathlib import Path

# === 1) Hàm load dữ liệu .npz ===
def load_npz_data(npz_path: Path):
    """Đọc u, v (pixel coords) và cam_xyz (Nx3) từ file .npz."""
    data = np.load(npz_path)
    return data['u'], data['v'], data['cam_xyz']

# === 2) Tạo depth map 2D (float32) ===
def build_depth_map(u: np.ndarray,
                    v: np.ndarray,
                    cam_xyz: np.ndarray,
                    H: int,
                    W: int,
                    fill_value: float = np.inf) -> np.ndarray:
    """
    Rasterize các điểm 3D xuống ảnh (H,W), giữ giá trị depth nhỏ nhất
    nếu nhiều điểm cùng rơi vào 1 pixel.
    fill_value nên là np.inf để clamp dễ hơn.
    """
    dists = np.linalg.norm(cam_xyz, axis=1)
    depth = np.full((H, W), fill_value, dtype=np.float32)

    for ui, vi, di in zip(u.astype(int), v.astype(int), dists):
        if 0 <= ui < W and 0 <= vi < H:
            if di < depth[vi, ui]:
                depth[vi, ui] = di

    depth[depth == np.inf] = 0.0
    return depth

# === 3) Chuẩn hóa lên 16-bit và clamp 100m ===
def normalize_depth(depth_map: np.ndarray,
                    max_dist: float = 100.0) -> np.ndarray:
    """
    - Clip depth về [0, max_dist]
    - Scale về [0 .. 65535], dtype=uint16
    """
    depth_clipped = np.clip(depth_map, 0.0, max_dist)
    depth_16 = np.round(depth_clipped / max_dist * 65535.0).astype(np.uint16)
    return depth_16

# === 4) Các phương pháp hoàn thiện depth ===
def fill_depth_inpaint(depth: np.ndarray) -> np.ndarray:
    """Inpaint (Navier–Stokes) trên depth 8-bit rồi scale ngược về 16-bit."""
    depth8 = np.clip(depth / 256, 0, 255).astype(np.uint8)
    mask   = (depth8 == 0).astype(np.uint8) * 255
    inpaint8 = cv2.inpaint(depth8, mask, inpaintRadius=5, flags=cv2.INPAINT_TELEA)
    return (inpaint8.astype(np.uint16) * 256)

# === 5) Hàm tạo dataset RGB-D (4-ch) với nội suy ===
def create_rgbd_dataset(
    image_dir: Path,
    npz_dir: Path,
    output_dir: Path,
    image_ext: str = ".png",
    max_dist: float = 100.0,
):
    """
    Với mỗi ảnh RGB <stem>.png và file điểm <stem>_data.npz,
    sinh ảnh 4-kênh <stem>_rgbd.png (B,G,R,depth16) với nội suy depth.
    """
    output_dir.mkdir(parents=True, exist_ok=True)

    for img_path in sorted(image_dir.glob(f"*{image_ext}")):
        stem     = img_path.stem
        npz_path = npz_dir / f"{stem}_data.npz"
        if not npz_path.exists():
            print(f"[!] Thiếu NPZ: {npz_path}, bỏ qua {stem}")
            continue

        # 1) Load RGB và point-cloud
        img       = cv2.imread(str(img_path), cv2.IMREAD_COLOR)  # BGR uint8
        H, W      = img.shape[:2]
        u, v, cz  = load_npz_data(npz_path)

        # 2) Build depth map float32
        depth_map = build_depth_map(u, v, cz, H, W, fill_value=np.inf)

        # 3) Normalize & quantize depth → uint16
        depth16 = normalize_depth(depth_map, max_dist=max_dist)

        # 4) Nội suy depth nếu cần
        depth16 = fill_depth_inpaint(depth16)

        # 5) Stack BGR(uint8) → uint16 và depth16
        bgr16  = img.astype(np.uint16)
        rgbd16 = np.dstack((bgr16, depth16))

        # 6) Lưu PNG 16-bit
        out_path = output_dir / f"{stem}_rgbd.png"
        cv2.imwrite(str(out_path), rgbd16)
        print(f"[✓] Saved RGB-D 16-bit : {out_path}")


In [2]:
# === 6) Ví dụ chạy cho train & test ===
if __name__ == "__main__":
    DATA_ROOT      = Path("/content/drive/MyDrive/Khóa Luận/Data_Object")
    IMG_TRAIN      = DATA_ROOT / "training/image_2"
    NPZ_TRAIN      = DATA_ROOT / "output_npz_training"
    OUT_RGBD_TRAIN = DATA_ROOT / "Code/Early_Fusion_RGBD_Yolov8/4chan/rgbd_images_inpaint_training"
    IMG_TEST       = DATA_ROOT / "testing/image_2"
    NPZ_TEST       = DATA_ROOT / "output_npz_testing"
    OUT_RGBD_TEST  = DATA_ROOT / "Code/Early_Fusion_RGBD_Yolov8/4chan/rgbd_images_inpaint_testing"

    # Chạy
    create_rgbd_dataset(IMG_TRAIN, NPZ_TRAIN,  OUT_RGBD_TRAIN)
    # create_rgbd_dataset(IMG_TEST,  NPZ_TEST,   OUT_RGBD_TEST)

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
[✓] Saved RGB-D 16-bit : /content/drive/MyDrive/Khóa Luận/Data_Object/Code/Early_Fusion_RGBD_Yolov8/4chan/rgbd_images_inpaint_training/002481_rgbd.png
[✓] Saved RGB-D 16-bit : /content/drive/MyDrive/Khóa Luận/Data_Object/Code/Early_Fusion_RGBD_Yolov8/4chan/rgbd_images_inpaint_training/002482_rgbd.png
[✓] Saved RGB-D 16-bit : /content/drive/MyDrive/Khóa Luận/Data_Object/Code/Early_Fusion_RGBD_Yolov8/4chan/rgbd_images_inpaint_training/002483_rgbd.png
[✓] Saved RGB-D 16-bit : /content/drive/MyDrive/Khóa Luận/Data_Object/Code/Early_Fusion_RGBD_Yolov8/4chan/rgbd_images_inpaint_training/002484_rgbd.png
[✓] Saved RGB-D 16-bit : /content/drive/MyDrive/Khóa Luận/Data_Object/Code/Early_Fusion_RGBD_Yolov8/4chan/rgbd_images_inpaint_training/002485_rgbd.png
[✓] Saved RGB-D 16-bit : /content/drive/MyDrive/Khóa Luận/Data_Object/Code/Early_Fusion_RGBD_Yolov8/4chan/rgbd_images_inpaint_training/002486_rgbd.png
[✓] Saved RGB-D 16-bit : /content/dri

In [ ]:
# Chạy
create_rgbd_dataset(IMG_TEST,  NPZ_TEST,   OUT_RGBD_TEST)

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
[✓] Saved RGB-D 16-bit : /content/drive/MyDrive/Khóa Luận/Data_Object/Code/Early_Fusion_RGBD_Yolov8/4chan/rgbd_images_inpaint_testing/002518_rgbd.png
[✓] Saved RGB-D 16-bit : /content/drive/MyDrive/Khóa Luận/Data_Object/Code/Early_Fusion_RGBD_Yolov8/4chan/rgbd_images_inpaint_testing/002519_rgbd.png
[✓] Saved RGB-D 16-bit : /content/drive/MyDrive/Khóa Luận/Data_Object/Code/Early_Fusion_RGBD_Yolov8/4chan/rgbd_images_inpaint_testing/002520_rgbd.png
[✓] Saved RGB-D 16-bit : /content/drive/MyDrive/Khóa Luận/Data_Object/Code/Early_Fusion_RGBD_Yolov8/4chan/rgbd_images_inpaint_testing/002521_rgbd.png
[✓] Saved RGB-D 16-bit : /content/drive/MyDrive/Khóa Luận/Data_Object/Code/Early_Fusion_RGBD_Yolov8/4chan/rgbd_images_inpaint_testing/002522_rgbd.png
[✓] Saved RGB-D 16-bit : /content/drive/MyDrive/Khóa Luận/Data_Object/Code/Early_Fusion_RGBD_Yolov8/4chan/rgbd_images_inpaint_testing/002523_rgbd.png
[✓] Saved RGB-D 16-bit : /content/drive/MyD

# **Chia train_valid**

In [3]:
import os, shutil
from pathlib import Path
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split

In [6]:
# ROOT trỏ vào folder chứa code 4chan của bạn
ROOT        = Path('/content/drive/MyDrive/Khóa Luận/Data_Object/Code/Early_Fusion_RGBD_Yolov8/4chan')

# Ảnh RGB-D và nhãn gốc
RGBD_TRAIN  = ROOT / 'rgbd_images_inpaint_training'
RGBD_TEST   = ROOT / 'rgbd_images_inpaint_testing'
LABELS_YOLO = Path('/content/drive/MyDrive/Khóa Luận/Data_Object/Code/Early_Fusion_RGBD_Yolov8/4chan/labels_4chan')

In [7]:
# Lấy list ảnh và nhãn
imgs = sorted(RGBD_TRAIN.glob('*_rgbd.png'))
lbls = sorted(LABELS_YOLO.glob('*_rgbd.txt'))
assert len(imgs) == len(lbls), "Số ảnh và nhãn không khớp!"

pairs = list(zip(imgs, lbls))
print(f"Tổng số cặp ảnh–nhãn: {len(pairs)}")

Tổng số cặp ảnh–nhãn: 7481


## **Chia train/val (90:10)**

In [8]:
# Split 9:1
train_pairs, val_pairs = train_test_split(
    pairs, test_size=0.1, shuffle=True, random_state=42
)
print(f"Train: {len(train_pairs)}  —  Val: {len(val_pairs)}")

Train: 6732  —  Val: 749


In [9]:
# Tạo thư mục local
TRAIN_IMG = Path('/content/train/images')
TRAIN_LBL = Path('/content/train/labels')
VAL_IMG   = Path('/content/valid/images')
VAL_LBL   = Path('/content/valid/labels')
for d in [TRAIN_IMG, TRAIN_LBL, VAL_IMG, VAL_LBL]:
    d.mkdir(parents=True, exist_ok=True)

In [10]:
# Copy file vào train/val
for img_path, lbl_path in tqdm(train_pairs, desc='Copy train'):
    shutil.copy(img_path, TRAIN_IMG/img_path.name)
    shutil.copy(lbl_path, TRAIN_LBL/lbl_path.name)

for img_path, lbl_path in tqdm(val_pairs, desc='Copy valid'):
    shutil.copy(img_path, VAL_IMG/img_path.name)
    shutil.copy(lbl_path, VAL_LBL/lbl_path.name)

Copy train:   0%|          | 0/6732 [00:00<?, ?it/s]

Copy valid:   0%|          | 0/749 [00:00<?, ?it/s]

### **zip**

In [11]:
TRAIN_DIR = Path('/content/train')
VALID_DIR = Path('/content/valid')

OUTPUT_ZIP = Path('/content/drive/MyDrive/Khóa Luận/Data_Object/Code/Early_Fusion_RGBD_Yolov8/4chan/train_valid_inpaint.zip')

import zipfile

# Tạo ZIP và thêm 2 folder train & valid
with zipfile.ZipFile(OUTPUT_ZIP, 'w', zipfile.ZIP_DEFLATED) as zf:
    # thêm train/*
    for file_path in TRAIN_DIR.rglob('*'):
        zf.write(file_path, file_path.relative_to('/content'))
    # thêm valid/*
    for file_path in VALID_DIR.rglob('*'):
        zf.write(file_path, file_path.relative_to('/content'))

print(f"[✓] Đã tạo file ZIP: {OUTPUT_ZIP}")

[✓] Đã tạo file ZIP: /content/drive/MyDrive/Khóa Luận/Data_Object/Code/Early_Fusion_RGBD_Yolov8/4chan/train_valid_inpaint.zip
